## Домашняя работа к лекции "Основы веб-скрапинга и работы с API"

### Задание 1
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: - -

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json
from datetime import datetime

In [9]:
KEYWORDS = ['Frontend', 'программист']
def get_articles(words, URL_1 = 'https://habr.com/ru/all/'):
    res = requests.get(URL_1)
    soup = BeautifulSoup(res.text, 'html.parser')
    news_blocks = soup.find_all('article', class_ = 'post post_preview')
    
    preview = pd.DataFrame()
    
    for news_block in news_blocks:
        for key in KEYWORDS:
            if key.lower() in news_block.text.lower():
                date = news_block.find('span', class_ = 'post__time').text
                title = news_block.find('h2', class_ = 'post__title').text
                link = news_block.find('a', class_ = 'post__title_link').get('href')
                all_info = {'date': date, 'title': title.strip(), 'link': link}
                preview = pd.concat([preview, pd.DataFrame([all_info])])
          
    preview = preview.drop_duplicates()
    return preview
get_articles(KEYWORDS)

,date,title,link
0,сегодня в 13:42,Переезд длиною в год: Украина – остров Мэн (ча...,https://habr.com/ru/post/536646/


### Задание 2.
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода: EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [38]:
EMAIL = ['xxx@x.ru', 'yyy@y.com', 'zzz@z.com']
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'Vaar-Header-App-Product': 'hackcheck-web-avast','Vaar-Version': '0'}
addresses = {'emailAddresses': EMAIL}

In [39]:
res = requests.post(url, json=addresses, headers=headers)
email_df = pd.DataFrame(res.json()['breaches']).T.reset_index() 
email_df = email_df[['site', 'description', 'publishDate']]
email_df['email'] = [', '.join(el.keys()) for el in list(res.json()['data'].values())]

email_df

,site,description,publishDate,email
0,ir.netease.com,"In October 2015, the Chinese internet and gami...",2016-11-07T00:00:00Z,zzz@z.com
1,dubsmash.com,"In December 2018, Dubsmash's database was alle...",2019-03-07T00:00:00Z,zzz@z.com
2,gamerzplanet.net,"In 2012, GamerzPlanet's user database was alle...",2017-03-23T00:00:00Z,zzz@z.com
3,tianya.cn,"In 2011, Tianya's user database was allegedly ...",2017-03-31T00:00:00Z,zzz@z.com
4,stratfor.com,"In December 2011, a group of hackers called An...",2016-10-23T00:00:00Z,zzz@z.com
5,chegg.com,"In 2018, education tech company Chegg's databa...",2018-08-24T00:00:00Z,zzz@z.com
6,wattpad.com,"In June 2020, the online writing community Wat...",2020-07-23T00:00:00Z,xxx@x.ru
7,azcentral.com,"At an unconfirmed date, online Arizona newspap...",2020-01-03T00:00:00Z,zzz@z.com
8,shein.com,"In June 2018, fast-fashion retailer Shein's da...",2019-05-16T00:00:00Z,zzz@z.com
9,wishbone.io,"In January 2020, the online poll website Wishb...",2020-05-28T00:00:00Z,zzz@z.com
